In [1]:
p=  "/Users/wfh/code/lc/langchain/docs/_dist/docs_skeleton/docs/integrations/chat_loaders/discord_chats.txt"

In [2]:
from langchain.chat_loaders.discord import DiscordChatLoader

loader = DiscordChatLoader(
    path=p, 
    user_name="talkingtower", # the user handle to represent the ai message
)

In [3]:
sessions = loader.load()

In [4]:
from langchain.adapters import openai as openai_adapter

openai_messages = openai_adapter.convert_messages_for_finetuning(sessions)

In [10]:
# Our chat is alternating, we will make each datapoint a group of 8 messages,
# with 4 messages overlapping
chunk_size = 8
overlap = 4
message_groups = [
    conversation_messages[i: i + chunk_size] 
    for conversation_messages in openai_messages
    for i in range(0, len(conversation_messages) - chunk_size + 1, chunk_size - overlap)
]

In [11]:
import json
import io

import openai


my_file = io.BytesIO()
for group in message_groups:
    my_file.write((json.dumps({"messages": group}) + "\n").encode('utf-8'))

my_file.seek(0)
training_file = openai.File.create(
  file=my_file,
  purpose='fine-tune'
)

In [13]:
job = openai.FineTuningJob.create(training_file=training_file.id, model="gpt-3.5-turbo")

In [14]:
import time
start = time.time()

while True:
  ftj = openai.FineTuningJob.retrieve(job.id)
  if ftj.fine_tuned_model is None:
    print(f"Waiting for fine-tuning to complete... Elapsed: {time.time() - start}", end="\r", flush=True)
    time.sleep(10)
  else:
    print(ftj.fine_tuned_model, flush=True)
    break

ft:gpt-3.5-turbo-0613:personal::7qtt96yblapsed: 476.21012806892395


In [15]:
openai.FineTuningJob.retrieve(job.id).status

'succeeded'

In [20]:
from langchain import chat_models, prompts

model_name = ftj.fine_tuned_model
# Example: ft:gpt-3.5-turbo-0613:personal::5mty86jblapsed
llm = chat_models.ChatOpenAI(model=model_name)
print(llm.predict("Hi i'm reporterbob"))

Hello Reporter Bob! How can I assist you today?
